<a href="https://colab.research.google.com/github/cloudjwissa/Project-2-Retail-Sales-Analytics-and-Real-Time-Demand-Forecasting/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#pyspark setup

!apt-get update
# Install Java 8 (required by Spark)
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Install Spark
!pip install pyspark

# setup environment variables
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

spark = SparkSession.builder.appName("Project2").getOrCreate()

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

output_dir = "output/"
task2_output_total = output_dir + "task2/task2_total.csv"
task2_output_avgsales = output_dir + "task2/task2_avgsales.csv"
task2_output_season = output_dir + "task2/task2_season.csv"
task3_output = output_dir + "task3/task3.csv"
task4_output = output_dir + "task4/task4.csv"

checkpoint_dir = "checkpoint/task5/"
task5_output = output_dir + "task5.csv"

In [3]:
train_df = spark.read.json("train.json")
reviwes_df = spark.read.json("test.json")
train_df.show()
#reviwes_df = reviwes_df.withColumn("helpful", F.col("helpful").cast("double"))
#reviwes_df.printSchema()
#print(reviwes_df.describe())
# Check if any rows have nulls in the 'helpful' column after casting
#invalid_rows = reviwes_df.filter(F.col("sentence").isNull())
# Show rows that could not be cast to double
#invalid_rows.show()

+----------+-------+--------------------+--------------------+--------------------+
|      asin|helpful|      main_image_url|       product_title|            sentence|
+----------+-------+--------------------+--------------------+--------------------+
|B000AO3L84|    1.7|http://ecx.images...|Canon 430EX Speed...|this flash is a s...|
|B001SEQPGK|    1.3|http://ecx.images...|Sony Cyber-shot D...|The pictures were...|
|0553386697|    1.9|http://ecx.images...|The Whole-Brain C...|A very good resou...|
|B006SUWZH2|   0.25|http://ecx.images...|Memorex Portable ...|We have it in a c...|
|B000W7F5SS|    0.9|http://ecx.images...|Harry Potter and ...|Again the makers ...|
|B000AO3L84|    2.0|http://ecx.images...|Canon 430EX Speed...|This flash is a g...|
|B00081NX5U|   0.73|http://ecx.images...|iPod Detachable R...|So I've had these...|
|B00000F1D3|    0.9|http://ecx.images...|             Believe|they're cd's or t...|
|B00000FCBH|    1.3|http://ecx.images...|  2Pac Greatest Hits|he proved that

In [4]:
reviwes_df.show()

+----------+-------+--------------------+--------------------+--------------------+
|      asin|helpful|      main_image_url|       product_title|            sentence|
+----------+-------+--------------------+--------------------+--------------------+
|B00VG90446|   1.07|http://ecx.images...|Flexion KS-902 Ki...|so it stays in pl...|
|B001196MG0|   1.33|http://ecx.images...|Savage 107X12-1 S...|Love this seamles...|
|B00081NX5U|   1.17|http://ecx.images...|iPod Detachable R...|very happy with m...|
|B003HC9JIW|    1.6|http://ecx.images...|Start! Walking At...|Even for someone ...|
|B00C30FCUI|   1.49|http://ecx.images...|Symphonized NRG P...|, those have alwa...|
|B001196MG0|   1.47|http://ecx.images...|Savage 107X12-1 S...|but after a year ...|
|B00AR1G3FS|   1.24|http://ecx.images...|Farewell Live Fro...|While not quite a...|
|B007R3AZNK|   0.67|http://ecx.images...|Driving Towards T...|Until now, Sloe G...|
|0761165975|    1.0|http://ecx.images...|The Wedding Plann...|I considered t

In [5]:
online_retail_df = spark.read.csv("Online-Retail.csv", inferSchema=True, header=True)

In [6]:
online_retail_df.select([F.count(F.when(col(c).isNull(), c)).alias(c) for c in online_retail_df.columns]).show()

# Drop rows with nulls in important columns like CustomerID, Description, etc.
online_retail_df = online_retail_df.dropna(subset=["CustomerID", "Description", "InvoiceDate", "Quantity", "UnitPrice"])
#online_retail_df.select([F.count(F.when(col(c).isNull(), c)).alias(c) for c in online_retail_df.columns]).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|        0|        0|       1454|       0|          0|        0|    135080|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



In [7]:
#drop duplicates
online_retail_df = online_retail_df.dropDuplicates()

In [8]:
#cast the invoice date to timestamp
online_retail_df = online_retail_df.withColumn("InvoiceDate", F.to_timestamp("InvoiceDate", "M/d/yyyy H:mm"))
online_retail_df = online_retail_df.withColumn("InvoiceDate", F.col("InvoiceDate").cast(TimestampType()))

In [9]:
#check for outliers
online_retail_df.filter((col("Quantity") < 0) | (col("UnitPrice") < 0)).show()

# Drop rows where Quantity or UnitPrice are negative (common outlier check)
online_retail_df = online_retail_df.filter((F.col("Quantity") >= 0) & (F.col("UnitPrice") >= 0))
#online_retail_df.filter((col("Quantity") < 0) | (col("UnitPrice") < 0)).show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|  C538350|   85099F|JUMBO BAG STRAWBERRY|      -2|2010-12-10 15:01:00|     1.65|     13798|United Kingdom|
|  C538375|    22220|CAKE STAND LOVEBI...|      -1|2010-12-12 11:19:00|     9.95|     17126|United Kingdom|
|  C539726|    22791|T-LIGHT GLASS FLU...|     -10|2010-12-21 14:24:00|     1.25|     17007|United Kingdom|
|  C540307|    22084|PAPER CHAIN KIT E...|     -36|2011-01-06 12:58:00|     2.95|     15823|United Kingdom|
|  C542138|    20866|BLUE ROSE FABRIC ...|    -120|2011-01-25 17:21:00|     1.06|     17368|United Kingdom|
|  C543347|    22629| SPACEBOY LUNCH BOX |      -1|2011-02-07 12:44:00|     1.95|     12472|       Germany|
|  C544830|    22059|CERAMIC

In [10]:
#standardize for consistent description characters
online_retail_df = online_retail_df.withColumn("Description", F.upper(F.col("Description")))

#handle special cases in case for better performance for MLlib
#online_retail_df = online_retail_df.filter(~F.col("InvoiceNo").startswith("C"))

In [11]:
online_retail_df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536367|    22745|POPPY'S PLAYHOUSE...|       6|2010-12-01 08:34:00|      2.1|     13047|United Kingdom|
|   536368|    22960|JAM MAKING SET WI...|       6|2010-12-01 08:34:00|     4.25|     13047|United Kingdom|
|   536388|    22915|ASSORTED BOTTLE T...|      12|2010-12-01 09:59:00|     0.42|     16250|United Kingdom|
|   536401|    21464|DISCO BALL ROTATO...|       1|2010-12-01 11:21:00|     4.25|     15862|United Kingdom|
|   536412|    22569|FELTCRAFT CUSHION...|       2|2010-12-01 11:49:00|     3.75|     17920|United Kingdom|
|   536425|    22645|CERAMIC HEART FAI...|      12|2010-12-01 12:08:00|     1.45|     13758|United Kingdom|
|   536488|    22376|AIRLINE

In [12]:
#-----------------------------------
#Task 2: Sales Data Aggregation and Feature Engineering
#-----------------------------------
#total sales per product per month
#get the month and year from invoicedate
online_retail_df = online_retail_df.withColumn("Month", F.month("InvoiceDate")).withColumn("Year", F.year("InvoiceDate"))

#make a revenue column, total quantity * price
online_retail_df = online_retail_df.withColumn("Revenue", col("Quantity") * col("UnitPrice"))

#total sales per product and month. calculated by summing total quantity * price
total_sales_df = online_retail_df.groupBy("StockCode", "Description", "Month", "Year").agg(F.sum("Revenue").alias("TotalSales"))
total_sales_df.show()

+---------+--------------------+-----+----+------------------+
|StockCode|         Description|Month|Year|        TotalSales|
+---------+--------------------+-----+----+------------------+
|    22692|DOORMAT WELCOME T...|    4|2011| 512.6999999999999|
|    84945|MULTI COLOUR SILV...|    5|2011| 791.5799999999999|
|    22384|LUNCH BAG PINK PO...|    1|2011| 902.1500000000001|
|    21221|SET/4 BADGES CUTE...|    2|2011|             43.75|
|   47591D|PINK FAIRY CAKE C...|    5|2011| 601.8000000000002|
|    22114|HOT WATER BOTTLE ...|   12|2010|1863.0500000000002|
|    22236|CAKE STAND 3 TIER...|    1|2011|403.04999999999995|
|    22407|MONEY BOX FIRST A...|    4|2011|             18.75|
|    20768| GREEN FERN JOURNAL |    7|2011|              35.7|
|    85213|MINI  ZINC GARDEN...|    3|2011|44.199999999999996|
|    20981|12 PENCILS TALL T...|    6|2011|25.499999999999996|
|    84828|JUNGLE POPSICLES ...|    7|2011|             112.5|
|    22624|IVORY KITCHEN SCALES|    3|2011|            

In [13]:
#average revenue per customer
#get average revnue for each customer id
avg_revnue_df = online_retail_df.groupBy("CustomerID").agg(F.avg("Revenue").alias("AverageRevenue")).orderBy(F.desc("AverageRevenue"))
avg_revnue_df.show()

+----------+------------------+
|CustomerID|    AverageRevenue|
+----------+------------------+
|     12346|           77183.6|
|     16446|           56157.5|
|     15098|           13305.5|
|     15749|           4453.43|
|     15195|            3861.0|
|     13135|            3096.0|
|     17846|            2033.1|
|     18087|2027.8599999999997|
|     16532|            1687.2|
|     16000|1377.0777777777778|
|     16754|            1001.2|
|     12755| 952.9874999999998|
|     18133| 931.4999999999999|
|     12798| 872.1299999999999|
|     17949| 835.8640000000001|
|     17553|             743.8|
|     15299| 643.8585714285715|
|     16308|             640.0|
|     16986|             624.4|
|     18080|            615.75|
+----------+------------------+
only showing top 20 rows



In [14]:
#seasonal patterns for top selling products
#get products by stock code and their highest total revnue as sum
top_products_df = online_retail_df.groupBy("StockCode").agg(F.sum("Revenue").alias("TotalRevenue")).orderBy(F.desc("TotalRevenue"))

#join with df to get montly data, group by product and month for each of their total revenue
seasonal_pattern = online_retail_df.join(top_products_df.limit(10), "StockCode").groupBy("StockCode", "Description", "Month").agg(F.sum("Revenue").alias("MonthlySales"))
seasonal_pattern.show()

+---------+--------------------+-----+------------------+
|StockCode|         Description|Month|      MonthlySales|
+---------+--------------------+-----+------------------+
|   85123A|WHITE HANGING HEA...|    4| 9581.649999999998|
|    84879|ASSORTED COLOUR B...|    1| 2704.189999999999|
|    23166|MEDIUM CERAMIC TO...|    9|            397.26|
|    47566|       PARTY BUNTING|    1|1815.1499999999999|
|        M|              MANUAL|    8|           2989.54|
|    23084|  RABBIT NIGHT LIGHT|    9|            235.14|
|   85123A|WHITE HANGING HEA...|    2| 4912.650000000001|
|   85099B|JUMBO BAG RED RET...|   10| 9763.060000000005|
|    22423|REGENCY CAKESTAND...|    1|10765.499999999998|
|   85099B|JUMBO BAG RED RET...|    7|            5654.6|
|     POST|             POSTAGE|   11|          10349.95|
|    47566|       PARTY BUNTING|   11|3715.7099999999996|
|   85123A|WHITE HANGING HEA...|   11|13849.929999999993|
|    47566|       PARTY BUNTING|    9|            4387.0|
|     POST|   

In [15]:
#customer liftime value: total revenue per customer
clv_df = online_retail_df.groupBy("CustomerID", "StockCode").agg(F.sum("Revenue").alias("CustomerLifetimeValue"))
clv_df.show()

+----------+---------+---------------------+
|CustomerID|StockCode|CustomerLifetimeValue|
+----------+---------+---------------------+
|     15363|    22382|                 16.5|
|     15498|    22749|                 15.0|
|     17530|    22557|   28.049999999999997|
|     17841|    22554|                57.75|
|     17730|   71101E|                 10.2|
|     17581|   85099B|                405.5|
|     16271|   85123A|                 64.9|
|     17235|   85184C|   35.400000000000006|
|     14156|    22927|                 71.4|
|     17501|    21977|   13.200000000000001|
|     15555|    21793|   11.850000000000001|
|     16327|    22355|                 32.3|
|     15382|    23282|                34.86|
|     13700|    22661|               144.25|
|     17454|    21931|                 19.5|
|     13113|    22423|                700.8|
|     16656|    21790|    777.5999999999999|
|     13933|    23198|                 17.4|
|     13451|    21179|                 2.34|
|     1449

In [16]:
#product popularity: counted by unique transaction made under each stock code
product_popularity = online_retail_df.groupBy("StockCode").agg(F.countDistinct("InvoiceNo").alias("PopularityScore"))
product_popularity.show()

+---------+---------------+
|StockCode|PopularityScore|
+---------+---------------+
|    21889|            449|
|    21259|            237|
|    22728|            613|
|    21452|            133|
|    22121|            114|
|    21894|             71|
|    90143|              7|
|    22596|            234|
|    84881|              5|
|    21248|             52|
|    22254|             41|
|    21249|             79|
|    23318|            329|
|   90026D|              3|
|    90022|              4|
|    21331|              7|
|    23459|             19|
|    20868|             31|
|   90210B|              6|
|    23843|              1|
+---------+---------------+
only showing top 20 rows



In [17]:
#seasonal trends:
#make season column based on month
online_retail_df = online_retail_df.withColumn("Season",
                    F.when(col("Month").isin(12, 1, 2), "Winter") #if month is in one of these numbers
                    .when(col("Month").isin(3, 4, 5), "Spring")
                    .when(col("Month").isin(6, 7, 8), "Summer")
                    .when(col("Month").isin(9, 10, 11), "Fall")
                    )

#total revenue of each product and season
seasonal_trends = online_retail_df.groupBy("StockCode", "Season").agg(F.sum("Revenue").alias("SeasonalSales"))
seasonal_trends.show()

+---------+------+------------------+
|StockCode|Season|     SeasonalSales|
+---------+------+------------------+
|    21110|Winter|            1349.3|
|    22668|Winter|            1411.7|
|    23203|Summer|15717.460000000015|
|    21213|Summer|1750.9500000000005|
|    21371|Summer|             129.0|
|    22728|Summer|           4546.26|
|    22966|Winter|2459.8599999999997|
|   90002D|Winter|              30.0|
|    22149|Spring|            2487.7|
|    22661|Summer|1333.0500000000002|
|    22089|Summer| 926.7500000000002|
|    37446|Winter| 659.5999999999999|
|    22252|Spring|             97.94|
|    22760|Winter|             849.9|
|   84558A|Winter|268.45000000000005|
|    22421|Winter|            159.48|
|    22301|Winter|1107.4499999999998|
|    22537|Spring|            190.26|
|    23147|Summer|1035.2999999999995|
|    16219|Summer|197.99999999999997|
+---------+------+------------------+
only showing top 20 rows



In [18]:
#write the aggregated data to the directory as csv files
#try:
#    total_sales_df.write.csv(task2_output_total, header=True)
#    avg_revnue_df.write.csv(task2_output_avgsales, header=True)
#    seasonal_pattern.write.csv(task2_output_season, header=True)
#except ValueError as e:
#    print(f"error {e}")

In [19]:
# Join features into a consolidated DataFrame
forecasting_df = total_sales_df.join(clv_df, "StockCode", "left") \
                               .join(product_popularity, "StockCode", "left") \
                               .join(seasonal_trends, "StockCode", "left")

# Fill nulls with 0 or appropriate values for ML training
forecasting_df = forecasting_df.fillna(0)

In [20]:
# Create lag features using window functions
from pyspark.sql.window import Window

windowSpec = Window.partitionBy("StockCode").orderBy("Year", "Month")
forecasting_df = forecasting_df.withColumn("Lag_1_Month", F.lag("TotalSales", 1).over(windowSpec))
forecasting_df = forecasting_df.withColumn("Lag_2_Months", F.lag("TotalSales", 2).over(windowSpec))

# Drop rows with null values created by lags
forecasting_df = forecasting_df.na.drop()

# Display the consolidated DataFrame structure
forecasting_df.show()

+---------+--------------------+-----+----+------------------+----------+---------------------+---------------+------+------------------+------------------+------------------+
|StockCode|         Description|Month|Year|        TotalSales|CustomerID|CustomerLifetimeValue|PopularityScore|Season|     SeasonalSales|       Lag_1_Month|      Lag_2_Months|
+---------+--------------------+-----+----+------------------+----------+---------------------+---------------+------+------------------+------------------+------------------+
|    10002|INFLATABLE POLITI...|   12|2010|190.39999999999998|     17967|                 0.85|             49|Winter| 519.3499999999999|190.39999999999998|190.39999999999998|
|    10002|INFLATABLE POLITI...|   12|2010|190.39999999999998|     17967|                 0.85|             49|Spring|180.20000000000002|190.39999999999998|190.39999999999998|
|    10002|INFLATABLE POLITI...|   12|2010|190.39999999999998|     17337|                  1.7|             49|Winter| 5

In [21]:
forecasting_df.printSchema()

root
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- TotalSales: double (nullable = false)
 |-- CustomerID: integer (nullable = true)
 |-- CustomerLifetimeValue: double (nullable = false)
 |-- PopularityScore: long (nullable = true)
 |-- Season: string (nullable = true)
 |-- SeasonalSales: double (nullable = false)
 |-- Lag_1_Month: double (nullable = true)
 |-- Lag_2_Months: double (nullable = true)



In [22]:
# Assemble feature columns
feature_cols = ["Lag_1_Month", "Lag_2_Months", "CustomerLifetimeValue", "PopularityScore", "SeasonalSales"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Scale the features
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        # Handles outliers or infinite values by replacing them with mean/median
                        withMean=True, withStd=True)

# Initialize the regression model (e.g., Linear Regression)
lr = LinearRegression(featuresCol="scaledFeatures", labelCol="TotalSales")

# Create a pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[assembler, scaler, lr])



In [23]:
from pyspark.sql.functions import when, isnan, col as pyspark_col
# Import necessary types for type checking
from pyspark.sql.types import IntegerType, FloatType, DoubleType

# Split data into training and testing sets
train_data, test_data = forecasting_df.randomSplit([0.8, 0.2], seed=42)

# Before fitting, ensure data types are correct and handle potential issues
# Check if columns have numerical types (e.g. int, float)
for column_name in feature_cols + ["TotalSales"]: # Changed 'col' to 'column_name'
    data_type = train_data.schema[column_name].dataType
    if not isinstance(data_type, (IntegerType, FloatType, DoubleType)):
        print(f"Column '{column_name}' has an unexpected type: {data_type}. Consider casting it to a numerical type.")

# Handle NaNs and infinite values
for column_name in feature_cols + ["TotalSales"]: # Changed 'col_name' to 'column_name'
    # Use pyspark.sql.functions.col to refer to the column
    # Avoid using 'col' as a variable name, use 'pyspark_col' instead
    # Rename the imported 'col' function to 'pyspark_col' to avoid conflicts
    train_data = train_data.withColumn(
        column_name,
        when(isnan(pyspark_col(column_name)) | (pyspark_col(column_name) == float("inf")) | (pyspark_col(column_name) == float("-inf")), 0).otherwise(pyspark_col(column_name))
    )



# Fit the model
model = pipeline.fit(train_data)

# Make predictions
predictions = model.transform(test_data)

# Evaluate the model
evaluator = RegressionEvaluator(labelCol="TotalSales", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})

print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")

Column 'PopularityScore' has an unexpected type: LongType(). Consider casting it to a numerical type.


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/errors/exceptions/captured.py", line 179, in deco
    return f(*a, **kw)
  File "/usr/local/lib/python3.10/dist-packages/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python

Py4JError: py4j.reflection.TypeUtil.isInstanceOf does not exist in the JVM